# Can we code a deep reasearch agent?(Basic Level)
# By - Aditya Bhatt

# TODO:

# 1.Ask the user what exactly they need a research on?(a simple follow up question)


# Enter the Research Topic

In [18]:
query="On what topic you want to do deep research on ?"
n=input(query)
print("User has asked for research on ",n)

User has asked for research on  Indian Cricket


# Install the necessary libraries


In [30]:
pip uninstall -q openai pydantic 

^C
Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install -q openai pydantic --upgrade

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: uvicorn 0.14.0 has a non-standard dependency specifier click>=7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of uvicorn or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Set the client

In [12]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()
try:
    client = AzureOpenAI(
                        api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
                        api_version = "2024-02-01",
                        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
                        )
    
    print("Client created successfully")
except Exception as e:
    print(e)
    print("Error in creating the client")


Client created successfully


# Test the client

In [13]:
try:
    response = client.chat.completions.create(
        model="mini-deployment", # model = "deployment_name".
        messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Hi?"}
    ]
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(e)
    print("Basic API call failed")


Hello! How can I assist you today?


# Convert the user query into a search query

# Concept used :Structured Output and Pydantic way of doing it


In [28]:
from pydantic import BaseModel
from openai import AzureOpenAI


class SearchQuery(BaseModel):
    """
    This is the search query that will be used to search the web
    """
    query: str

prompt1 = f"You need to convert the user query into a search query (which will be used to search the web) for the user query: {n}"

print(prompt1)
completion = client.beta.chat.completions.parse(
    model="mini-deployment",  # replace with the model deployment name of your gpt-4o 2024-08-06 deployment
    messages=[
        {"role": "system", "content": "You are a helpful assistant that converts user queries into search queries."},
        {"role": "user", "content": prompt1},
    ],
    response_format=SearchQuery,
)

search_query = completion.choices[0].message.parsed
print(search_query)
print(completion.model_dump_json(indent=2))


You need to convert the user query into a search query (which will be used to search the web) for the user query: Indian Cricket


ImportError: cannot import name 'LazyClassAttribute' from 'pydantic._internal._utils' (c:\Users\aditya\AppData\Local\Programs\Python\Python39\lib\site-packages\pydantic\_internal\_utils.py)

# Once we have the search query , we need a plan to break down the query into smaller pieces which will be used to define a report

In [22]:
class Plan(BaseModel):
    """
    This will breakdown the query into smaller pieces which will be used to define a report (word file, or pdf file (not sure yet))
    """
    items: list[str]  # A list of items separated by commas

prompt2 = '''
As a research assistant, your task is to thoroughly break down the user query into detailed, manageable topics that will serve as the foundation for a comprehensive report. Ensure that each topic is relevant and contributes to a well-structured analysis.

Example: User query: {Indian Food}\n\nReturn the following topics: [Overview of Indian cuisine, Historical influences on Indian food, Regional variations in Indian cuisine, Cultural significance of Indian food, Contemporary trends in Indian dining, Health aspects of Indian food]
'''
print(prompt2)

completion = client.chat.completions.create(
    model="mini-deployment",  
    messages=[
        {"role": "system", "content": "You are a research assistant that breaks down user queries into detailed, manageable topics."},
        {"role": "user", "content": prompt2},
    ],
    response_model=Plan,
)







As a research assistant, your task is to thoroughly break down the user query into detailed, manageable topics that will serve as the foundation for a comprehensive report. Ensure that each topic is relevant and contributes to a well-structured analysis.

Example: User query: {Indian Food}

Return the following topics: [Overview of Indian cuisine, Historical influences on Indian food, Regional variations in Indian cuisine, Cultural significance of Indian food, Contemporary trends in Indian dining, Health aspects of Indian food]

